## 1

What is the approximate depth of a Decision Tree trained(without restrictions) on a training set with 1 million instance?



$log_2(10^2)=23$

## 2

Is node's Gini impurity generally lower or greater than its parent's? It is generally lower/greater, or always lower/greater?


$O(n \times mlog(m)$, it will take $\frac{10mlog(10m)}{mlog(m)}$ hours.

## 3

If a Decision Tree is overfitting the training set, is it good idea to try decresing max\_depth?


Yes

## 4

If a Decision Tree is underfitting the training set, is it good idea to try scaling the input features?


No

## 5

In [ ]:
If it take one hour to train a Decision tree on the training set containing 1 million instances. roughly how much time will it take to train another Decision Tree on a training set contain 10 million instances?

$O(n \times mlog(m)$, it will take $\frac{10mlog(10m)}{mlog(m)}$ hours.

## 6 

If your training set contains 100,000 instances, will setting presort=True speed up the training?

No

# 7

Follow these steps to train and fine-tune a Decision Tree for the moons dataset:

a. To build a moons dataset, use make moons(n samples=10000, noise=0.4).

b. Divide the dataset into a training and a test collection with train test split().

c. To find good hyperparameters values for a DecisionTreeClassifier, use grid search with cross-
validation (with the GridSearchCV class). Try different values for max leaf nodes.

d. Use these hyperparameters to train the model on the entire training set, and then assess its
output on the test set. You can achieve an accuracy of 85 to 87 percent.

In [1]:
from sklearn.datasets import make_moons
dataset = make_moons(n_samples=10000, noise=0.4)

In [2]:
X = dataset[0]
y = dataset[1]

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
params = [
    {'max_leaf_nodes': [5, 7, 10, 13, 15]}
]
grid_search = GridSearchCV(DecisionTreeClassifier(), params, verbose=2, n_jobs=-1, cv=3)

In [5]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] max_leaf_nodes=5 ................................................
[CV] max_leaf_nodes=5 ................................................
[CV] max_leaf_nodes=7 ................................................
[CV] max_leaf_nodes=5 ................................................
[CV] ................................. max_leaf_nodes=5, total=   0.0s
[CV] max_leaf_nodes=7 ................................................
[CV] ................................. max_leaf_nodes=7, total=   0.0s
[CV] ................................. max_leaf_nodes=5, total=   0.0s
[CV] max_leaf_nodes=7 ................................................
[CV] ................................. max_leaf_nodes=5, total=   0.0s
[CV] ................................. max_leaf_nodes=7, total=   0.0s
[CV] max_leaf_nodes=10 ...............................................
[CV] max_leaf_nodes=10 ...............................................
[CV] max_leaf_nod

[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.2s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'max_leaf_nodes': [5, 7, 10, 13, 15]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [6]:
grid_search.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=5, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [7]:
from sklearn.metrics import accuracy_score
y_pred = grid_search.best_estimator_.predict(X_test)

In [8]:
accuracy_score(y_test, y_pred)

0.85050000000000003

# 8 

Follow these steps to grow a forest:

a. Using the same method as before, create 1,000 subsets of the training set, each containing
100 instances chosen at random. You can do this with Scikit-ShuffleSplit Learn&#39;s class.

b. Using the best hyperparameter values found in the previous exercise, train one Decision
Tree on each subset. On the test collection, evaluate these 1,000 Decision Trees. These Decision

Trees would likely perform worse than the first Decision Tree, achieving only around 80% accuracy,
since they were trained on smaller sets.

c. Now the magic begins. Create 1,000 Decision Tree predictions for each test set case, and
keep only the most common prediction (you can do this with SciPy&#39;s mode() function). Over the test
collection, this method gives you majority-vote predictions.

d. On the test range, evaluate these predictions: you should achieve a slightly higher accuracy
than the first model (approx 0.5 to 1.5 percent higher). You&#39;ve successfully learned a Random Forest
classifier!

In [9]:
datasets = make_moons(n_samples=10000, noise=0.2)

In [10]:
X = datasets[0]
y = datasets[1]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [12]:
import numpy as np
np.random.seed(42)
m,_ = X_train.shape

In [13]:
n_subsets = 1000
n_instance = 100
tree_clfs = []
for _ in range(n_subsets):
    shuffle_inx = np.random.randint(m, size=n_instance)
    X_train_i = X_train[shuffle_inx]
    y_train_i = y_train[shuffle_inx]
    tree_clf = DecisionTreeClassifier(max_leaf_nodes=10)
    tree_clf.fit(X_train_i, y_train_i)
    tree_clfs.append(tree_clf)

In [14]:
accuaries = []
for tree_cls in tree_clfs:
    y_pred = tree_clf.predict(X_test)
    accuaries.append(accuracy_score(y_pred, y_test))

In [20]:
np.mean(accuaries)

0.90799999999999992

In [21]:
from collections import Counter
m_test, n_features = X_test.shape
y_pred = []
for i in range(m_test):
    if i % 100 == 0:
        print(i/m_test)
    instance = X_test[i:i+1,:]
    counter = Counter([clf.predict(instance)[0] for clf in tree_clfs])
    y_pred.append(counter.most_common(1)[0][0])

0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95


In [24]:
accuracy_score(y_pred, y_test)

0.96150000000000002